In [383]:
import numpy as np
from skimage import measure, morphology
from scipy import signal
import time
import math

In [415]:
color_codes = np.array(['0', 'J', 'R', 'G', 'B', 'Y', 'P']) # Mask for converting field to code representation
fallable_nums = np.array([1, 2, 3, 4, 5, 6])
no_garbage = np.array([0, 0, 2, 3, 4, 5, 6]) # Mask for seeing the field without garbage
PUYO_TYPE = {
    'NONE': 0,
    'GARBAGE': 1,
    'RED': 2,
    'GREEN': 3,
    'BLUE': 4,
    'YELLOW': 5,
    'PURPLE': 6
}

In [416]:
field = np.array([[0, 0, 0, 0, 0, 0],
                  [0, 0, 0, 0, 0, 0],
                  [0, 0, 0, 0, 0, 0],
                  [0, 0, 0, 0, 0, 0],
                  [0, 0, 0, 0, 0, 0],
                  [0, 0, 0, 0, 0, 0],
                  [0, 0, 5, 0, 0, 0],
                  [0, 1, 5, 0, 0, 0],
                  [0, 5, 5, 0, 0, 0],
                  [3, 5, 5, 1, 1, 6],
                  [3, 2, 4, 5, 6, 6],
                  [3, 3, 2, 4, 5, 5],
                  [2, 2, 4, 4, 5, 6]], dtype=np.uint8)
print(field.shape)
print(color_codes[field])

(13, 6)
[['0' '0' '0' '0' '0' '0']
 ['0' '0' '0' '0' '0' '0']
 ['0' '0' '0' '0' '0' '0']
 ['0' '0' '0' '0' '0' '0']
 ['0' '0' '0' '0' '0' '0']
 ['0' '0' '0' '0' '0' '0']
 ['0' '0' 'Y' '0' '0' '0']
 ['0' 'J' 'Y' '0' '0' '0']
 ['0' 'Y' 'Y' '0' '0' '0']
 ['G' 'Y' 'Y' 'J' 'J' 'P']
 ['G' 'R' 'B' 'Y' 'P' 'P']
 ['G' 'G' 'R' 'B' 'Y' 'Y']
 ['R' 'R' 'B' 'B' 'Y' 'P']]
[[0 0 0 0 0 0]
 [0 0 0 0 0 0]
 [0 0 0 0 0 0]
 [0 0 0 0 0 0]
 [0 0 0 0 0 0]
 [0 0 0 0 0 0]
 [0 0 5 0 0 0]
 [0 0 5 0 0 0]
 [0 5 5 0 0 0]
 [3 5 5 0 0 6]
 [3 2 4 5 6 6]
 [3 3 2 4 5 5]
 [2 2 4 4 5 6]]


## Check colored Puyo pops

In [417]:
# Calculated connected components
labels = measure.label(no_garbage[field], background=0, connectivity=1)
print(labels)

[[ 0  0  0  0  0  0]
 [ 0  0  0  0  0  0]
 [ 0  0  0  0  0  0]
 [ 0  0  0  0  0  0]
 [ 0  0  0  0  0  0]
 [ 0  0  0  0  0  0]
 [ 0  0  1  0  0  0]
 [ 0  0  1  0  0  0]
 [ 0  1  1  0  0  0]
 [ 2  1  1  0  0  3]
 [ 2  4  5  6  3  3]
 [ 2  2  7  8  9  9]
 [10 10  8  8  9 11]]


In [418]:
# Groups that are big enough to pop
groups, count = np.unique(labels[labels != 0], return_counts=True)
pop_labels = groups[count >= 4]
pop_counts = count[count >= 4]
np.array([pop_labels, pop_counts]).transpose()

array([[1, 6],
       [2, 4]])

In [419]:
# Number of colors popping
unique = np.unique(field[np.isin(labels, pop_labels)])
color_count = unique.shape[0]
print(color_count)

2


In [420]:
# Get where the colors are popping
color_pop_mask = np.isin(labels, pop_labels)
print(color_pop_mask)

[[False False False False False False]
 [False False False False False False]
 [False False False False False False]
 [False False False False False False]
 [False False False False False False]
 [False False False False False False]
 [False False  True False False False]
 [False False  True False False False]
 [False  True  True False False False]
 [ True  True  True False False False]
 [ True False False False False False]
 [ True  True False False False False]
 [False False False False False False]]


## Check Garbage Puyo Pops

In [443]:
garbages = np.argwhere(np.isin(field, PUYO_TYPE['GARBAGE']))
garbage_to_clear = []
for i, pos in enumerate(garbages):
    y, x = pos
    if ((y > 1 and color_pop_mask[y - 1, x] == True) or (y < 12 and color_pop_mask[y + 1, x] == True) or (x > 0 and color_pop_mask[y, x - 1] == True) or (x < 6 and color_pop_mask[y, x + 1] == True)):
        garbage_to_clear.append(i)

garbages[garbage_to_clear]

field[to_clear[:, 0], to_clear[:, 1]]

array([1, 1], dtype=uint8)

In [436]:
popping_puyos = np.argwhere(color_pop_mask)

[[ 6  2]
 [ 7  2]
 [ 8  1]
 [ 8  2]
 [ 9  0]
 [ 9  1]
 [ 9  2]
 [10  0]
 [11  0]
 [11  1]]


In [109]:
# Binary dilation to get garbage Puyos that are next to popping groups
garbage_mask = morphology.binary_dilation(color_pop_mask) * np.isin(field, PUYO_TYPE['GARBAGE'])
print(garbage_mask)

[[False False False False False False]
 [False False False False False False]
 [False False False False False False]
 [False False False False False False]
 [False False False False False False]
 [False False False False False False]
 [False False False False False False]
 [False  True False False False False]
 [False False False False False False]
 [False False False  True False False]
 [False False False False False False]
 [False False False False False False]
 [False False False False False False]]


## Check for any Puyos that need to drop

In [211]:
field = np.array([[0, 1, 0, 1, 0, 0],
                  [0, 5, 0, 0, 0, 0],
                  [0, 0, 3, 0, 0, 0],
                  [0, 0, 4, 0, 0, 1],
                  [0, 0, 1, 0, 0, 1],
                  [0, 0, 0, 0, 0, 1],
                  [0, 0, 0, 0, 0, 1],
                  [0, 0, 0, 0, 0, 3],
                  [0, 0, 1, 0, 0, 4],
                  [0, 0, 0, 1, 1, 6],
                  [3, 2, 4, 5, 6, 6],
                  [3, 3, 2, 4, 5, 5],
                  [2, 2, 4, 4, 5, 6]], dtype=np.uint8)

fallable_field = np.isin(field, fallable_nums).astype(np.int)
fallable_field[fallable_field == 0] = -1
print(fallable_field)

[[-1  1 -1  1 -1 -1]
 [-1  1 -1 -1 -1 -1]
 [-1 -1  1 -1 -1 -1]
 [-1 -1  1 -1 -1  1]
 [-1 -1  1 -1 -1  1]
 [-1 -1 -1 -1 -1  1]
 [-1 -1 -1 -1 -1  1]
 [-1 -1 -1 -1 -1  1]
 [-1 -1  1 -1 -1  1]
 [-1 -1 -1  1  1  1]
 [ 1  1  1  1  1  1]
 [ 1  1  1  1  1  1]
 [ 1  1  1  1  1  1]]


In [212]:
has_drop_kernel = np.array([[1], [-1]])
has_drop_kernel

array([[ 1],
       [-1]])

In [216]:
corr = signal.correlate2d(fallable_field, has_drop_kernel)[1:]
has_drop = 2 in corr
print(corr)
print(has_drop)
# This has an interesting property.
# 2 = puyos that need to drop
# -2 = empty cells just above the surface of the stack

[[ 0  0  0  2  0  0]
 [ 0  2 -2  0  0  0]
 [ 0  0  0  0  0 -2]
 [ 0  0  0  0  0  0]
 [ 0  0  2  0  0  0]
 [ 0  0  0  0  0  0]
 [ 0  0  0  0  0  0]
 [ 0  0 -2  0  0  0]
 [ 0  0  2 -2 -2  0]
 [-2 -2 -2  0  0  0]
 [ 0  0  0  0  0  0]
 [ 0  0  0  0  0  0]
 [ 1  1  1  1  1  1]]
True


## Apply Drops

In [353]:
test_field = np.copy(field)
since = time.time()
for i in range(6):
    col = test_field[:, i]
    not_zero = col[col != 0]
    # test_field[:, i] = np.concatenate((col[col == 0], col[col != 0]))
    test_field[:, i] = 0
    test_field[(13 - not_zero.shape[0]):, i] = not_zero
print(time.time() - since)
print(test_field)

0.00017189979553222656
[[0 0 0 0 0 0]
 [0 0 0 0 0 0]
 [0 0 0 0 0 0]
 [0 0 0 0 0 1]
 [0 0 0 0 0 1]
 [0 0 0 0 0 1]
 [0 0 3 0 0 1]
 [0 0 4 0 0 3]
 [0 1 1 1 0 4]
 [0 5 1 1 1 6]
 [3 2 4 5 6 6]
 [3 3 2 4 5 5]
 [2 2 4 4 5 6]]


## Apply Pops

In [413]:
field = np.array([[0, 0, 0, 0, 0, 0],
                  [0, 0, 0, 0, 0, 0],
                  [0, 0, 0, 0, 0, 0],
                  [0, 0, 0, 0, 0, 0],
                  [0, 0, 0, 0, 0, 0],
                  [3, 0, 0, 0, 0, 0],
                  [0, 0, 5, 0, 0, 0],
                  [0, 1, 5, 0, 0, 0],
                  [0, 5, 5, 0, 0, 0],
                  [3, 5, 5, 1, 1, 6],
                  [3, 2, 4, 5, 6, 6],
                  [3, 3, 2, 4, 5, 5],
                  [2, 2, 4, 4, 5, 6]], dtype=np.uint8)

pop_field = np.copy(field)
pop_field[color_pop_mask + garbage_mask] = 0
print(pop_field)
print(color_codes[field])

[[0 0 0 0 0 0]
 [0 0 0 0 0 0]
 [0 0 0 0 0 0]
 [0 0 0 0 0 0]
 [0 0 0 0 0 0]
 [3 0 0 0 0 0]
 [0 0 0 0 0 0]
 [0 0 0 0 0 0]
 [0 0 0 0 0 0]
 [0 0 0 0 1 6]
 [0 2 4 5 6 6]
 [0 0 2 4 5 5]
 [2 2 4 4 5 6]]
[['0' '0' '0' '0' '0' '0']
 ['0' '0' '0' '0' '0' '0']
 ['0' '0' '0' '0' '0' '0']
 ['0' '0' '0' '0' '0' '0']
 ['0' '0' '0' '0' '0' '0']
 ['G' '0' '0' '0' '0' '0']
 ['0' '0' 'Y' '0' '0' '0']
 ['0' 'J' 'Y' '0' '0' '0']
 ['0' 'Y' 'Y' '0' '0' '0']
 ['G' 'Y' 'Y' 'J' 'J' 'P']
 ['G' 'R' 'B' 'Y' 'P' 'P']
 ['G' 'G' 'R' 'B' 'Y' 'Y']
 ['R' 'R' 'B' 'B' 'Y' 'P']]


In [404]:
# Game constants
CHAIN_POWER = [0, 8, 16, 32, 64, 96, 128, 160, 192, 224, 256, 288, 320, 352, 384, 416, 448, 480, 512, 544, 576, 608, 640, 672]
COLOR_BONUS = [0, 3, 6, 12, 24]
GROUP_BONUS = np.ones(100) * 10
GROUP_BONUS[:7] = np.array([0, 2, 3, 4, 5, 6, 7])

# Constants for simulator
color_codes = np.array(['0', 'J', 'R', 'G', 'B', 'Y', 'P']) # Mask for converting field to code representation
fallable_nums = np.array([1, 2, 3, 4, 5, 6])
no_garbage = np.array([0, 0, 2, 3, 4, 5, 6]) # Mask for seeing the field without garbage
PUYO_TYPE = {
    'NONE': 0,
    'GARBAGE': 1,
    'RED': 2,
    'GREEN': 3,
    'BLUE': 4,
    'YELLOW': 5,
    'PURPLE': 6
}
has_drop_kernel = np.array([[1], [-1]])

def _needs_drop(field: np.ndarray):
    fallable_field = np.isin(field, fallable_nums).astype(np.int)
    fallable_field[fallable_field == 0] = -1
    corr = signal.correlate2d(fallable_field, has_drop_kernel)[1:]
    return 2 in corr

def _apply_drop(field: np.ndarray):
    for i in range(6):
        col = field[:, i]
        not_zero = col[col != 0]
        field[:, i] = 0
        field[(field.shape[0] - not_zero.shape[0]):, i] = not_zero
    return field

def _analyze_pops(field: np.ndarray, puyo_to_pop=4):
    # Calculated connected components
    labels = measure.label(no_garbage[field], background=0, connectivity=1)

    # Groups that are big enough to pop
    groups, count = np.unique(labels[labels != 0], return_counts=True)
    pop_labels = groups[count >= puyo_to_pop]
    pop_counts = count[count >= puyo_to_pop]

    # Has pop? Boolean variable
    has_pops = pop_labels.shape[0] > 0

    # Number of colors popping
    unique = np.unique(field[np.isin(labels, pop_labels)])
    color_count = unique.shape[0]

    # Get where the colors are popping
    color_pop_mask = np.isin(labels, pop_labels)

    # Binary dilation to get garbage Puyos that are next to popping groups
    garbage_mask = morphology.binary_dilation(color_pop_mask) * np.isin(field, PUYO_TYPE['GARBAGE'])

    return has_pops, pop_counts, color_count, color_pop_mask, garbage_mask

def _calculate_step_score(step, pop_counts, color_count, puyo_to_pop):
    # Calculate Puyo popped
    puyos_popped = np.sum(pop_counts)
    
    # Calculate Group Bonus
    pop_count_ind = pop_counts - puyo_to_pop
    group_bonus = np.sum(GROUP_BONUS[pop_count_ind])


    # Calculate Color Bonus
    color_bonus = COLOR_BONUS[color_count - 1]

    # Get Chain Power
    chain_power = CHAIN_POWER[step - 1]
    return (10 * puyos_popped) * np.clip((chain_power + color_bonus + group_bonus), 1, 999)

def simulate_chain(field: np.ndarray, step=0, puyo_to_pop=4, score=0, target_point=70):
    # Check for drops
    needs_drop = _needs_drop(field)

    if needs_drop:
        field = _apply_drop(field)
    
    # Check for pops
    has_pops, pop_counts, color_count, color_pop_mask, garbage_mask = _analyze_pops(field, puyo_to_pop)

    if has_pops:
        current_step = step + 1 # Increase chain length
        step_score = _calculate_step_score(current_step, pop_counts, color_count, puyo_to_pop)
        total_score = score + step_score
        field[color_pop_mask + garbage_mask] = 0
        return simulate_chain(field, step=current_step, puyo_to_pop=puyo_to_pop, score=total_score, target_point=target_point)
    else:
        damage = math.floor(score / target_point)
        return field, step, score, damage

In [445]:
%timeit new_field, step, score, damage = simulate_chain(np.copy(field))
print('Chain Length: ', step)
print('Chain Score: ', score)
print('Garbage Sent: ', damage)

1.68 ms ± 18.2 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
Chain Length:  5
Chain Score:  5680.0
Garbage Sent:  81
